In [ ]:
import cudf
from cuml.tsa.arima import ARIMA

import pandas as pd
import matplotlib
matplotlib.use('WebAgg')
import matplotlib.pyplot as plt
from math import sqrt
from multiprocessing import cpu_count
from joblib import Parallel
from joblib import delayed
from itertools import product
import warnings
from warnings import catch_warnings
from warnings import filterwarnings
import datetime
from ast import literal_eval
import numpy as np

filterwarnings('ignore')

# split a univariate dataset into train/test sets
def train_test_split(data, n_test):
	return data[:-n_test], data[-n_test:]

def cal_duration(time1, time2):
    td = time2 - time1
    return td.total_seconds()

if __name__ == '__main__':
    # load dataset
    df = pd.read_csv('eurusd_h1_2016_1_2021_10.csv')
    df= pd.DataFrame(df,columns = ['Close'])
    df = df.values
    df = df.flatten()
    print(df[0:10])

    #Calculate the natural logarithm of the data so that the first difference 
    #gets the data percentage growth.
    df = np.log(df)
    # for i in range(2, 50):
    i = 1
    n_test = 24
    # all_data = df[i * n_test:( i * n_test) + 3000]
    all_data = df[0:100]
    data = all_data[:-n_test] # For test of prediction last predict_step steps
  
    order = (2, 1, 2)
    s_order = (2, 1, 2, 30)
    time1 = datetime.datetime.now()

    # Build Model to test the result with the data_test
    model_fit = ARIMA(data, order=order, seasonal_order=s_order).fit()

    prediction = model_fit.forecast(n_test)

    # data=np.exp(data)
    actual=np.exp(all_data[-n_test:len(all_data)])
    prediction= np.exp(prediction)
    time2 = datetime.datetime.now()
    print('------------duration-----------')
    print(cal_duration(time1, time2))

    #Plot prediction n steps ahead
    plt.plot(actual, label='actual')
    plt.plot(prediction, label='prediction')
    plt.title('Prediction vs Actual')
    plt.legend(loc='upper left', fontsize=8)
    plt.savefig('{}test.png'.format(i))
    plt.close()
    # plt.show()